In [ ]:
pip install catboost

In [ ]:
	
! pip install lightgbm

In [ ]:
# Importing libraries
from numpy import loadtxt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.calibration import calibration_curve
from scipy.sparse import vstack
from xgboost import plot_tree
from xgboost import plot_importance
import matplotlib.pyplot as plt
import matplotlib.pyplot as pyplot
import pandas as pd
import numpy as np
from tabulate import tabulate
import pickle
import seaborn as sns
#import tensorflow as tf
import keras
from collections import Counter
%matplotlib inline
sns.set()

In [ ]:
#import data
data2 = pd.read_excel('joineddf2.xlsx')

data2.index = np.arange(1,28786,1)

data2 = data2.drop(columns=['Column1','id_assessment'])
# this drops all duplicated columns
data2 = data2.loc[:,~data2.columns.duplicated()]

data2 = data2[data2.columns[~data2.columns.isin(['id_student'])]]
# remove date




In [ ]:
#Reading the first few rows of your data
data2.head()

,code_module_x,code_presentation_x,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_submitted,is_banked,score,code_module_y,code_presentation_y,date_registration,date_unregistration
1,AAA,2013J,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,18.0,0.0,78.0,AAA,2013J,-159.0,NaN
2,AAA,2013J,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,22.0,0.0,70.0,AAA,2013J,-53.0,NaN
3,AAA,2013J,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,NaN,NaN,NaN,AAA,2013J,-92.0,12.0
4,AAA,2013J,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,17.0,0.0,72.0,AAA,2013J,-52.0,NaN
5,AAA,2013J,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,26.0,0.0,69.0,AAA,2013J,-176.0,NaN


In [ ]:
data2.isna().sum()

code_module_x               0
code_presentation_x         0
gender                      0
region                      0
highest_education           0
imd_band                 4103
age_band                    0
num_of_prev_attempts        0
studied_credits             0
disability                  0
final_result                0
date_submitted           5416
is_banked                5416
score                    5455
code_module_y               0
code_presentation_y         0
date_registration          38
date_unregistration     19817
dtype: int64

In [ ]:
data2['code_presentation_x']

1        2013J
2        2013J
3        2013J
4        2013J
5        2013J
         ...  
28781    2014J
28782    2014J
28783    2014J
28784    2014J
28785    2014J
Name: code_presentation_x, Length: 28785, dtype: object

In [ ]:
# lets do label encoding 
cat_columns=['code_module_x','code_presentation_x','code_presentation_x',
                            'gender' ,'region','highest_education','imd_band','age_band',
                            'num_of_prev_attempts','disability','final_result',
                            'is_banked','code_module_y','code_presentation_y']
for col in cat_columns:
  data2[col] = data2[col].astype('category')

data2['is_banked'].dtype.name



'category'

In [ ]:
data3 = data2

In [ ]:
cat_columns=['code_module_x','code_presentation_x','code_presentation_x',
                            'gender' ,'region','highest_education','imd_band','age_band',
                            'num_of_prev_attempts','disability','final_result',
                            'is_banked','code_module_y','code_presentation_y']

# we now assign a value that is strictly categorical
for col in cat_columns:
  if data2[col].dtype.name == 'category':
      data2[col] = data2[col].cat.codes
#obj_df["body_style_cat"] = obj_df["body_style"].cat.codes


In [ ]:
data3.dtypes

code_module_x              int8
code_presentation_x        int8
gender                     int8
region                     int8
highest_education          int8
imd_band                   int8
age_band                   int8
num_of_prev_attempts       int8
studied_credits           int64
disability                 int8
final_result               int8
date_submitted          float64
is_banked                  int8
score                   float64
code_module_y              int8
code_presentation_y        int8
date_registration       float64
date_unregistration     float64
dtype: object

In [ ]:
data2.head()

,code_module_x,code_presentation_x,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_submitted,is_banked,score,code_module_y,code_presentation_y,date_registration,date_unregistration
1,0,1,1,0,1,8,2,0,240,0,2,18.0,0,78.0,0,1,-159.0,NaN
2,0,1,0,6,1,1,1,0,60,0,2,22.0,0,70.0,0,1,-53.0,NaN
3,0,1,0,5,0,2,1,0,60,1,3,NaN,-1,NaN,0,1,-92.0,12.0
4,0,1,0,7,0,4,1,0,60,0,2,17.0,0,72.0,0,1,-52.0,NaN
5,0,1,0,11,2,4,0,0,60,0,2,26.0,0,69.0,0,1,-176.0,NaN


In [ ]:
X = data2[data2.columns[~data2.columns.isin(['final_result','date_registration',
                                             'date_submitted','date_unregistration'])]]
y =data2['final_result']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
X_train.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [ ]:
X_train = X_train.loc[:,~X_train.columns.duplicated()]

In [ ]:
# cat boost decision trees
import numpy as np

from catboost import CatBoostClassifier, Pool

cat_boost = CatBoostClassifier(iterations=3,
                           depth=2,
                           learning_rate=1,
                           loss_function='MultiClass',
                           verbose=True)

# train the model
cat_boost.fit(X_train, y_train)


0:	learn: 1.1251280	total: 14.2ms	remaining: 28.5ms
1:	learn: 1.1042061	total: 26.3ms	remaining: 13.1ms
2:	learn: 1.0753500	total: 38.5ms	remaining: 0us


In [ ]:
y_pred_cat = cat_boost.predict(X_test)

#Calculating the accuracy of the trained model

print(classification_report(y_test,y_pred_cat))
print(accuracy_score(y_test, y_pred_cat))

              precision    recall  f1-score   support

           0       0.31      0.01      0.01       548
           1       0.36      0.20      0.26      1241
           2       0.50      0.84      0.62      2116
           3       0.70      0.56      0.62      1852

    accuracy                           0.53      5757
   macro avg       0.47      0.40      0.38      5757
weighted avg       0.52      0.53      0.49      5757

0.5323953447976376


In [ ]:
#Ranking the features using the score
headers = ["name", "score"]
values = sorted(zip(X.columns, cat_boost.feature_importances_), key=lambda X: X[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                    score
score                 76.0541
code_module_x         23.9459
code_presentation_x    0
gender                 0
region                 0
highest_education      0
imd_band               0
age_band               0
num_of_prev_attempts   0
studied_credits        0
disability             0
is_banked              0
code_module_y          0
code_presentation_y    0


In [ ]:
from xgboost import XGBClassifier
xg_boost = XGBClassifier()
xg_boost.fit(X_train, y_train)

y_pred_xg = xg_boost.predict(X_test)


#Calculating the accuracy of the trained model

print(classification_report(y_test,y_pred_xg))
print(accuracy_score(y_test, y_pred_xg))

              precision    recall  f1-score   support

           0       0.55      0.04      0.07       548
           1       0.48      0.16      0.24      1241
           2       0.51      0.92      0.66      2116
           3       0.75      0.60      0.67      1852

    accuracy                           0.57      5757
   macro avg       0.57      0.43      0.41      5757
weighted avg       0.58      0.57      0.51      5757

0.5702622893868334


In [ ]:
#Ranking the features using the score
headers = ["name", "score"]
values = sorted(zip(X.columns, xg_boost.feature_importances_), key=lambda X: X[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                      score
score                 0.415981
is_banked             0.108279
code_module_x         0.0911567
studied_credits       0.0779072
highest_education     0.0560495
num_of_prev_attempts  0.05229
disability            0.0505681
code_presentation_x   0.0437117
gender                0.0385485
age_band              0.0228974
imd_band              0.0223305
region                0.0202802
code_module_y         0
code_presentation_y   0


In [ ]:
# light gradient boost
import lightgbm as lgb

lg_boost = lgb.LGBMClassifier()
lg_boost.fit(X_train,y_train)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_pred_lg = lg_boost.predict(X_test)


#Calculating the accuracy of the trained model

print(classification_report(y_test,y_pred_lg))
print(accuracy_score(y_test, y_pred_lg))

              precision    recall  f1-score   support

           0       0.50      0.11      0.18       548
           1       0.48      0.19      0.27      1241
           2       0.52      0.89      0.66      2116
           3       0.73      0.62      0.67      1852

    accuracy                           0.58      5757
   macro avg       0.56      0.45      0.44      5757
weighted avg       0.58      0.58      0.53      5757

0.5752996352266806


In [ ]:
#Ranking the features using the score
headers = ["name", "score"]
values = sorted(zip(X.columns, lg_boost.feature_importances_), key=lambda X: X[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                    score
score                    3127
region                   1725
code_module_x            1533
imd_band                 1396
studied_credits          1077
code_presentation_x       853
highest_education         822
age_band                  381
gender                    367
num_of_prev_attempts      327
disability                224
is_banked                 168
code_module_y               0
code_presentation_y         0


For models that need no null values

Imputed dataset

In [ ]:
data3.head()

,code_module_x,code_presentation_x,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_submitted,is_banked,score,code_module_y,code_presentation_y,date_registration,date_unregistration
1,0,1,1,0,1,8,2,0,240,0,2,18.0,0,78.0,0,1,-159.0,NaN
2,0,1,0,6,1,1,1,0,60,0,2,22.0,0,70.0,0,1,-53.0,NaN
3,0,1,0,5,0,2,1,0,60,1,3,NaN,-1,NaN,0,1,-92.0,12.0
4,0,1,0,7,0,4,1,0,60,0,2,17.0,0,72.0,0,1,-52.0,NaN
5,0,1,0,11,2,4,0,0,60,0,2,26.0,0,69.0,0,1,-176.0,NaN


In [ ]:
# lets impute 

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype category are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype.name == 'category' else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

data3 = DataFrameImputer().fit_transform(data2)

In [ ]:
data3.head()
# this confirms that all the categorical columns are still being written as categorical

,code_module_x,code_presentation_x,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_submitted,is_banked,score,code_module_y,code_presentation_y,date_registration,date_unregistration
1,0,1,1,0,1,8,2,0,240,0,2,18.00000,0,78.000000,0,1,-159.0,49.101695
2,0,1,0,6,1,1,1,0,60,0,2,22.00000,0,70.000000,0,1,-53.0,49.101695
3,0,1,0,5,0,2,1,0,60,1,3,25.58308,-1,72.753622,0,1,-92.0,12.000000
4,0,1,0,7,0,4,1,0,60,0,2,17.00000,0,72.000000,0,1,-52.0,49.101695
5,0,1,0,11,2,4,0,0,60,0,2,26.00000,0,69.000000,0,1,-176.0,49.101695


In [ ]:
# Check for null values
data3.isna().sum()

code_module_x           0
code_presentation_x     0
gender                  0
region                  0
highest_education       0
imd_band                0
age_band                0
num_of_prev_attempts    0
studied_credits         0
disability              0
final_result            0
date_submitted          0
is_banked               0
score                   0
code_module_y           0
code_presentation_y     0
date_registration       0
date_unregistration     0
dtype: int64

In [ ]:
# Checks if there are any null values
np.any(np.isnan(data3))

False

In [ ]:
# checks if there are infinite values
np.all(np.isfinite(data3))
# there are no infinite values in the imputed data,

True

In [ ]:
# lets reset index in data to try and get rid of 

data3 = data3.reset_index()

In [ ]:
X_imp = data3[data3.columns[~data3.columns.isin(['final_result'])]]
y_imp =data3['final_result']

In [ ]:
from sklearn.model_selection import train_test_split

X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(X_imp, y_imp, test_size=0.2, random_state=123)

In [ ]:
# Random forest
rf = RandomForestClassifier(n_estimators=10,random_state=42)                          
# Fit on training data
rf.fit(X_train_imp, y_train_imp)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred_rf = rf.predict(X_test_imp)

#Calculating the accuracy of the trained model

print(classification_report(y_test_imp,y_pred_rf))
print(accuracy_score(y_test_imp, y_pred_rf))

              precision    recall  f1-score   support

           0       0.43      0.25      0.32       548
           1       0.59      0.50      0.54      1241
           2       0.62      0.74      0.67      2116
           3       1.00      0.99      0.99      1852

    accuracy                           0.72      5757
   macro avg       0.66      0.62      0.63      5757
weighted avg       0.71      0.72      0.71      5757

0.7217300677436165


In [ ]:
#Ranking the features using the score
headers = ["name", "score"]
values = sorted(zip(X.columns, rf.feature_importances_), key=lambda X: X[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                       score
code_presentation_y   0.102139
code_module_x         0.0905429
score                 0.0809154
highest_education     0.0553632
age_band              0.0518375
code_module_y         0.0469552
disability            0.026582
imd_band              0.0217108
num_of_prev_attempts  0.0132415
region                0.0119149
gender                0.0113075
code_presentation_x   0.0105462
studied_credits       0.00843968
is_banked             0.00722082


In [ ]:
# KNN

from sklearn.neighbors import KNeighborsClassifier

knn_clf=KNeighborsClassifier()

knn_clf.fit(X_train_imp,y_train_imp)

y_pred_knn = knn_clf.predict(X_test_imp) #These are the predicted output values

#Calculating the accuracy of the trained model

print(classification_report(y_test_imp,y_pred_knn))
print(accuracy_score(y_test_imp, y_pred_knn))

              precision    recall  f1-score   support

           0       0.24      0.20      0.22       548
           1       0.34      0.37      0.36      1241
           2       0.49      0.63      0.55      2116
           3       0.95      0.62      0.75      1852

    accuracy                           0.53      5757
   macro avg       0.51      0.46      0.47      5757
weighted avg       0.58      0.53      0.54      5757

0.5310057321521626


In [ ]:
# light gradient boost
import lightgbm as lgb

lg_boost = lgb.LGBMClassifier()
lg_boost.fit(X_train_imp,y_train_imp)

y_pred_lg = lg_boost.predict(X_test_imp)


#Calculating the accuracy of the trained model

print(classification_report(y_test_imp,y_pred_lg))
print(accuracy_score(y_test_imp, y_pred_lg))

              precision    recall  f1-score   support

           0       0.55      0.12      0.19       548
           1       0.74      0.44      0.55      1241
           2       0.62      0.90      0.73      2116
           3       1.00      0.99      1.00      1852

    accuracy                           0.75      5757
   macro avg       0.73      0.61      0.62      5757
weighted avg       0.76      0.75      0.73      5757

0.7545596664929651


In [ ]:
from xgboost import XGBClassifier
xg_boost = XGBClassifier()
xg_boost.fit(X_train_imp, y_train_imp)

y_pred_xg = xg_boost.predict(X_test_imp)


#Calculating the accuracy of the trained model

print(classification_report(y_test_imp,y_pred_xg))
print(accuracy_score(y_test_imp, y_pred_xg))

              precision    recall  f1-score   support

           0       0.74      0.06      0.12       548
           1       0.78      0.40      0.53      1241
           2       0.61      0.93      0.74      2116
           3       1.00      0.99      1.00      1852

    accuracy                           0.75      5757
   macro avg       0.78      0.60      0.60      5757
weighted avg       0.78      0.75      0.72      5757

0.7547333680736494


In [ ]:
# Support Vector Machines, radial basis f

from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', random_state = 0)
svm.fit(X_train_imp, y_train_imp)

y_pred_svm = svm.predict(X_test_imp)


#Calculating the accuracy of the trained model

print(classification_report(y_test_imp,y_pred_svm))
print(accuracy_score(y_test_imp, y_pred_svm))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       548
           1       0.00      0.00      0.00      1241
           2       0.40      0.67      0.50      2116
           3       0.40      0.48      0.44      1852

    accuracy                           0.40      5757
   macro avg       0.20      0.29      0.23      5757
weighted avg       0.28      0.40      0.32      5757

0.40055584505819003


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Decision Trees

from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
dec_tree.fit(X_train_imp, y_train_imp)

y_pred_dt = dec_tree.predict(X_test_imp)


#Calculating the accuracy of the trained model

print(classification_report(y_test_imp,y_pred_dt))
print(accuracy_score(y_test_imp, y_pred_dt))


              precision    recall  f1-score   support

           0       0.29      0.27      0.28       548
           1       0.51      0.51      0.51      1241
           2       0.62      0.63      0.62      2116
           3       0.99      0.99      0.99      1852

    accuracy                           0.68      5757
   macro avg       0.60      0.60      0.60      5757
weighted avg       0.68      0.68      0.68      5757

0.684905332638527


In [ ]:
from catboost import CatBoostClassifier, Pool

cat_boost = CatBoostClassifier(iterations=3,
                           depth=2,
                           learning_rate=1,
                           loss_function='MultiClass',
                           verbose=True)

# train the model
cat_boost.fit(X_train_imp, y_train_imp)

y_pred_cat = cat_boost.predict(X_test_imp)

#Calculating the accuracy of the trained model

print(classification_report(y_test_imp,y_pred_cat))
print(accuracy_score(y_test_imp, y_pred_cat))



0:	learn: 0.7545885	total: 14.5ms	remaining: 29.1ms
1:	learn: 0.6891807	total: 28.3ms	remaining: 14.1ms
2:	learn: 0.6594307	total: 41.7ms	remaining: 0us
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       548
           1       0.95      0.18      0.31      1241
           2       0.57      1.00      0.73      2116
           3       1.00      0.98      0.99      1852

    accuracy                           0.72      5757
   macro avg       0.63      0.54      0.51      5757
weighted avg       0.74      0.72      0.65      5757

0.7219037693243009


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# naive bayes
from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(X_train_imp, y_train_imp)

y_pred_naive = naive.predict(X_test_imp)


#Calculating the accuracy of the trained model

print(classification_report(y_test_imp,y_pred_naive))
print(accuracy_score(y_test_imp, y_pred_naive))


              precision    recall  f1-score   support

           0       0.24      0.02      0.04       548
           1       0.70      0.22      0.33      1241
           2       0.58      0.97      0.73      2116
           3       0.99      0.95      0.97      1852

    accuracy                           0.71      5757
   macro avg       0.63      0.54      0.52      5757
weighted avg       0.71      0.71      0.65      5757

0.712002779225291


In [ ]:
# neural net

import numpy as np
import pandas as pd
import tensorflow as tf
#innitialise model
ann = tf.keras.models.Sequential()
# input and first layer
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
#2nd layer
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
# output layer
ann.add(tf.keras.layers.Dense(units=4, activation='sigmoid'))

In [ ]:
# training the model
# since we dont have one hot encoding and the target variable is multi-class labelled
ann.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train_imp, y_train_imp, batch_size = 32, epochs = 100)


Epoch 1/100
720/720 [==============================] - 1s 1ms/step - loss: 196.3511 - accuracy: 0.2517
Epoch 2/100
720/720 [==============================] - 1s 1ms/step - loss: 4.5734 - accuracy: 0.3124
Epoch 3/100
720/720 [==============================] - 1s 1ms/step - loss: 3.8607 - accuracy: 0.3337
Epoch 4/100
720/720 [==============================] - 1s 1ms/step - loss: 4.1896 - accuracy: 0.3518
Epoch 5/100
720/720 [==============================] - 1s 1ms/step - loss: 3.5240 - accuracy: 0.3819
Epoch 6/100
720/720 [==============================] - 1s 1ms/step - loss: 3.6007 - accuracy: 0.3882
Epoch 7/100
720/720 [==============================] - 1s 1ms/step - loss: 2.3422 - accuracy: 0.4528
Epoch 8/100
720/720 [==============================] - 1s 1ms/step - loss: 2.6482 - accuracy: 0.4568
Epoch 9/100
720/720 [==============================] - 1s 1ms/step - loss: 3.4832 - accuracy: 0.4629
Epoch 10/100
720/720 [==============================] - 1s 1ms/step - loss: 2.3329 - accu

Feature scaling, for numerical variables


In [ ]:
data4 = data3 

In [ ]:
data4.dtypes

index                     int64
code_module_x              int8
code_presentation_x        int8
gender                     int8
region                     int8
highest_education          int8
imd_band                   int8
age_band                   int8
num_of_prev_attempts       int8
studied_credits           int64
disability                 int8
final_result               int8
date_submitted          float64
is_banked                  int8
score                   float64
code_module_y              int8
code_presentation_y        int8
date_registration       float64
date_unregistration     float64
dtype: object

In [ ]:
import joblib


joblib.dump(xg_boost,'xg_boost_pss.pkl',compress=0)
joblib.dump(naive,'naive_pss.pkl',compress=0)
joblib.dump(lg_boost,'lg_boost_pss.pkl',compress=0)
joblib.dump(knn_clf,'knn_clf_pss.pkl',compress=0)
joblib.dump(cat_boost,'cat_boost_pss.pkl',compress=0)
#joblib.dump(ann,'ann_pss.pkl',compress=0) # TypeError: can't pickle weakref objects
joblib.dump(dec_tree,'dec_tree_pss,pkl',compress=0)


['dec_tree_pss,pkl']